In [21]:
import os
import pandas as pd
import glob

# show all columns
pd.options.display.max_columns = None

In [22]:
if not os.path.exists('Outputs'):
    os.makedirs('Outputs')
    
outputs = ['.\\Outputs', "scratch.gdb", 'results.gdb']
gdb = os.path.join(outputs[0], outputs[1])
gdb2 = os.path.join(outputs[0], outputs[2])

In [23]:
centers_eq = pd.read_csv(r".\Inputs\Centers_Parcel_EQ.csv")
centers_eq_ids = centers_eq['parcel_id'].to_list()
remm_folder = r'D:\Josh_Projects\REMM2_For_Python3_Internal_Use_6'

In [24]:
center_summary = pd.DataFrame()

pm_folder = os.path.join(remm_folder, 'REMMRun\\Progression_Metrics')
csvs = glob.glob(os.path.join(pm_folder, 'run_*_year_*_parcel_progression_metrics.csv'))
csvs = [csv for csv in csvs if 'base' not in csv]

for csv in csvs:
    df = pd.read_csv(csv)
     
    year = os.path.basename(csv).split('_')[3]

     # separate into inside and outside of center
    in_center = df[df['parcel_id'].isin(centers_eq_ids)==True].copy()
    out_center = df[df['parcel_id'].isin(centers_eq_ids)==False].copy()

    # new jobs (centered)
    in_gov_edu_added = in_center['jobs_gov_edu_added'].sum()
    in_jobs_health_added = in_center['jobs_health_added'].sum()
    in_jobs_accom_food_added = in_center['jobs_accom_food_added'].sum()
    in_jobs_manuf_added = in_center['jobs_manuf_added'].sum()
    in_jobs_office_added = in_center['jobs_office_added'].sum()
    in_jobs_other_added = in_center['jobs_other_added'].sum()
    in_jobs_retail_added = in_center['jobs_retail_added'].sum()
    in_jobs_wholesale_added = in_center['jobs_wholesale_added'].sum()

    in_new_retail_jobs = in_jobs_accom_food_added + in_jobs_retail_added
    in_new_industrial_jobs = in_jobs_manuf_added + in_jobs_wholesale_added
    in_new_office_jobs = in_jobs_other_added + in_jobs_office_added + in_jobs_health_added + in_gov_edu_added
    in_new_jobs = in_new_retail_jobs + in_new_industrial_jobs + in_new_office_jobs

    # new jobs (non-centered)
    out_gov_edu_added = out_center['jobs_gov_edu_added'].sum()
    out_jobs_health_added = out_center['jobs_health_added'].sum()
    out_jobs_accom_food_added = out_center['jobs_accom_food_added'].sum()
    out_jobs_manuf_added = out_center['jobs_manuf_added'].sum()
    out_jobs_office_added = out_center['jobs_office_added'].sum()
    out_jobs_other_added = out_center['jobs_other_added'].sum()
    out_jobs_retail_added = out_center['jobs_retail_added'].sum()
    out_jobs_wholesale_added = out_center['jobs_wholesale_added'].sum()

    out_new_retail_jobs = out_jobs_accom_food_added + out_jobs_retail_added
    out_new_industrial_jobs = out_jobs_manuf_added + out_jobs_wholesale_added
    out_new_office_jobs = out_jobs_other_added + out_jobs_office_added + out_jobs_health_added + out_gov_edu_added
    out_new_jobs = out_new_retail_jobs + out_new_industrial_jobs + out_new_office_jobs

    # filter to new residential
    in_center_new_sf = in_center[((in_center['was_developed']==1) | (in_center['was_redeveloped']==1)) & ((in_center['is_sf']==1))]
    in_center_new_mf = in_center[((in_center['was_developed']==1) | (in_center['was_redeveloped']==1)) & ((in_center['is_mf']==1))]
    out_center_new_sf = out_center[((out_center['was_developed']==1) | (out_center['was_redeveloped']==1)) & ((out_center['is_sf']==1))]
    out_center_new_mf = out_center[((out_center['was_developed']==1) | (out_center['was_redeveloped']==1)) & ((out_center['is_mf']==1))]

    # get the totals & counts
    in_new_sf_units = in_center_new_sf['residential_units'].sum()
    in_new_mf_units = in_center_new_mf['residential_units'].sum()
    out_new_sf_units = out_center_new_sf['residential_units'].sum()
    out_new_mf_units = out_center_new_mf['residential_units'].sum()
    in_new_units = in_new_sf_units + in_new_mf_units
    out_new_units = out_new_sf_units + out_new_mf_units

#     in_new_sqft = in_center_new_res['building_sqft'].sum()
#     out_new_sqft = out_center_new_res['building_sqft'].sum()
#     in_new_bldgs = in_center_new_res['parcel_id'].count()
#     out_new_bldgs = out_center_new_res['parcel_id'].count()

    # append current data to final dataframe 
    d = {
         'year': [year], 
         'in_new_res_units': [in_new_units], 'out_new_res_units': [out_new_units], 
         'in_new_sf_units': [in_new_sf_units], 'out_new_sf_units': [out_new_sf_units],
         'in_new_mf_units': [in_new_mf_units], 'out_new_mf_units': [out_new_mf_units],
         'in_new_jobs':[in_new_jobs], 'out_new_jobs':[out_new_jobs],
         'in_new_retail_jobs':[in_new_retail_jobs], 'out_new_retail_jobs':[out_new_retail_jobs], 
         'in_new_industrial_jobs':[in_new_industrial_jobs],'out_new_industrial_jobs':[out_new_industrial_jobs], 
         'in_new_office_jobs':[in_new_office_jobs], 'out_new_office_jobs':[out_new_office_jobs]
         }
#     d = {'year': [int(year)], 'in_new_units': [in_new_units], 'out_new_units': [out_new_units],'in_new_sqft': [in_new_sqft],'out_new_sqft': [out_new_sqft],'in_new_bldgs': [in_new_bldgs], 'out_new_bldgs': [out_new_bldgs],
#          'in_new_retail_jobs':[in_new_retail_jobs], 'out_new_retail_jobs':[out_new_retail_jobs], 'in_new_industrial_jobs':[in_new_industrial_jobs],'out_new_industrial_jobs':[out_new_industrial_jobs], 
#          'in_new_office_jobs':[in_new_office_jobs], 'out_new_office_jobs':[out_new_office_jobs], 'in_new_jobs':[in_new_jobs], 'out_new_jobs':[out_new_jobs]}

    row = pd.DataFrame(data=d)
    center_summary = pd.concat([center_summary, row])

In [25]:
# export
center_summary2 = center_summary.astype(int)
center_summary2 = center_summary2[center_summary['year'] != 2019]
center_summary2.to_csv(os.path.join(outputs[0], 'centered_vs_noncentered_growth_mq6.csv'), index=False)